In [1]:
# import h3
import geopandas as gpd
import rasterio
from rasterio.enums import Resampling
import rasterstats as rs
from shapely.geometry import Polygon
from rasterio.features import rasterize
from rasterio.transform import from_origin

In [2]:
def reclassify_raster(input_raster, output_raster):
    # Otwieranie pliku rasterowego
    with rasterio.open(input_raster) as src:
        data = src.read(1)  # Wczytanie pierwszej warstwy

        # Reklasyfikacja danych
        data[data == 2] = 0
        data[data == 1] = 1  # Zamiana wartości 2 na 1

        # Zapis nowego pliku z zaktualizowanymi danymi
        with rasterio.open(
            output_raster,
            'w',
            driver='GTiff',
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            dst.write(data, 1)


def zonal_stats(raster_path, hex_gdf, field):   
    with rasterio.open(raster_path) as raster_interpolate:
        # Masked = True sets no data values to np.nan if they are in the metadata
        raster_data = raster_interpolate.read(1, masked=True)
        raster_meta = raster_interpolate.profile

    hexagon_zonal = rs.zonal_stats(hex_gdf,
                                    raster_data,
                                    nodata=-999,
                                    affine=raster_meta['transform'],
                                    geojson_out=True,
                                    copy_properties=True,
                                    stats="count min mean max median")

    hexagons_zonal = gpd.GeoDataFrame.from_features(hexagon_zonal)
    hexagons_zonal[field] = hexagons_zonal['mean']
    # hexagons_zonal = hexagons_zonal[['geometry', field]]
    return hexagons_zonal


def fix_false_built_up(output_raster_path, output_raster_path_v2, vector_layer):
    # Wczytanie rastrowego pliku, który będzie modyfikowany
    with rasterio.open(output_raster_path) as src:
        meta = src.meta.copy()
        data = src.read(1)  # Wczytanie pierwszej warstwy

        # Tworzenie maski
        mask = rasterize(shapes=vector_layer.geometry, out_shape=data.shape, fill=0, 
                         transform=src.transform, all_touched=True, dtype='uint8')

        # Ustawienie wartości w danych na 0 tam, gdzie maska ma wartości 1
        data[mask == 1] = 0

        # Zapis zmodyfikowanego rastrowego pliku
        with rasterio.open(output_raster_path_v2, 'w', **meta) as dst:
            dst.write(data, 1)


def calculate_new_column(year, prev_year):
    if year >= prev_year * 1.2 and year >= prev_year + 0.1:
        return year
    else:
        return prev_year


# def load_gdf_and_generate_h3(file_path, resolution=9):
#     # Wczytanie GeoDataFrame
#     gdf = gpd.read_file(file_path)

#     # Lista na przechowywanie heksagonów H3
#     hexagons = []

#     # Przetwarzanie każdego poligonu w GDF
#     for index, row in gdf.iterrows():
#         # Uzyskanie geometrii każdego poligonu
#         poly = row['geometry']

#         # Generowanie heksagonów H3 dla danego poligonu
#         hex_ids = h3.polyfill(poly.__geo_interface__, resolution, geo_json_conformant=True)

#         # Tworzenie poligonów z heksagonów
#         hex_polygons = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in hex_ids]

#         # Dodawanie do listy
#         hexagons.extend(hex_polygons)

#     # Tworzenie nowego GeoDataFrame
#     hex_gdf = gpd.GeoDataFrame(geometry=hexagons)

#     return hex_gdf

In [ ]:
# # Przykład użycia
# file_path = r'd:\GIS\GIS_projects\2024_jakarta\johor\johor_admin.gpkg'
# hex_gdf = load_gdf_and_generate_h3(file_path)
# print(hex_gdf)

In [15]:
hex_gdf = gpd.read_file(r'D:\marcinszwagrzyk.github.io\data\hexes\hexes_empty.shp')
# Wczytanie danych wektorowych
vector_layer = gpd.read_file(r"D:\marcinszwagrzyk.github.io\data\training_fields\bledy.shp")

In [6]:
# Przykład użycia funkcji reklas
for year in [2016, 2018, 2020, 2022, 2024]:
    input_raster_path = fr'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\{year}_21042024_11classes.tif'
    output_raster_path = fr'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\{year}_21042024_11classes_rekl.tif'
    output_raster_path_v2 = fr'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\{year}_21042024_11classes_rekl_v2.tif'
    reclassify_raster(input_raster_path, output_raster_path)
    fix_false_built_up(output_raster_path, output_raster_path_v2, vector_layer)
    hex_gdf = zonal_stats(output_raster_path_v2, hex_gdf, year)

In [ ]:
# # TODO time series analysis
# przepusci wzrost tylko wtedy jesli mamy wiecej niz 10 pp
# albo wiecje niz 50% zmiany
# albo wiecej niz 50% ale nie mniej niz 10 pp
# TODO - for initial point - make a median of 2016 and 2018?
## - if 2016 bigger than 2018 - then apply mean of 2016 and 2018, then process

# TODO
# greenfield - dodatkowa kategoria- od zera jak jest cokolwiek pewnego

# TODO
# wymyslec jakas sensowna metryke na timeseries
#    # - np pow heksa *1/10 lub * 1/4 lub * 1/2 lub * 1 - w zaleznosci ile % zabudowy w heksie
# inne podejscie: zabudowany lub nie : > 15% ?

# co mozna robic? wyczyscic recznie rastry - klasyfikacje nr 1 i nr ostatnia
# daty posrednie - postarac sie uchwycic zmiane tempa zabudowy
# zliczanie powierzchni excavation YoY

In [10]:
hex_gdf

,geometry,min,max,mean,count,median,2016,2018,2020,2022,2024,2016_corr
0,"POLYGON ((103.58602 1.44552, 103.58557 1.44738...",0.0,1.0,0.378649,1199,0.0,0.432027,0.447873,0.489575,0.522102,0.378649,0.432027
1,"POLYGON ((103.75101 1.51502, 103.75056 1.51688...",0.0,1.0,0.491243,1199,0.0,0.663053,0.677231,0.790659,0.735613,0.491243,0.663053
2,"POLYGON ((104.01260 1.53511, 104.01215 1.53697...",0.0,0.0,0.000000,1203,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"POLYGON ((103.85551 1.51230, 103.85506 1.51417...",0.0,1.0,0.606489,1202,1.0,0.657238,0.648918,0.665557,0.674709,0.606489,0.657238
4,"POLYGON ((103.87671 1.55349, 103.87626 1.55536...",0.0,0.0,0.000000,1200,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
8664,"POLYGON ((103.71124 1.54342, 103.71079 1.54529...",0.0,1.0,0.614488,1201,1.0,0.665279,0.697752,0.736886,0.661116,0.614488,0.665279
8665,"POLYGON ((103.98012 1.59810, 103.97967 1.59997...",0.0,0.0,0.000000,1201,0.0,0.074938,0.000000,0.000000,0.000000,0.000000,0.074938
8666,"POLYGON ((103.82093 1.54799, 103.82048 1.54986...",0.0,1.0,0.557309,1204,1.0,0.730066,0.736711,0.738372,0.762458,0.557309,0.730066
8667,"POLYGON ((103.60762 1.40646, 103.60717 1.40833...",0.0,1.0,0.010824,1201,0.0,0.000833,0.000000,0.000833,0.012490,0.010824,0.000833


In [11]:
hex_gdf['2016_corr'] = hex_gdf[2016]

for year, prev_year in zip([2018, 2020, 2022, 2024], [2016,2018,2020,2022]):
    print(year, prev_year)
    # hex_gdf['2024_v2'] = hex_gdf.apply(calculate_new_column, axis=1)
    hex_gdf[f'{year}_corr'] = hex_gdf.apply(lambda row: calculate_new_column(row[year], row[f'{prev_year}_corr']), axis=1)
    

2018 2016
2020 2018
2022 2020
2024 2022


In [12]:
hex_gdf
hex_gdf[['geometry', 
         '2016_corr', '2018_corr', '2020_corr', '2022_corr', '2024_corr']]. \
    to_file(r'D:\marcinszwagrzyk.github.io\data\hexes\h3_all_crs.shp')